# Web Scraping


## Load required libraries

In [104]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import re

## HTTP Request

In [139]:
def getListOfRealEstateByPage(page):
    website = 'https://www.dibeo.at/obj/kauf/v?page='+str(page)+'&sort=stateFrom,desc&locationSuggestions=9:SUB_COUNTRY&size=10'
    headers = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.54'})
    return requests.get(website, headers=headers)


In [142]:
# Exctract only the number
def getNumber(str):
    return float(''.join([i for i in str if i.isdigit()]))

In [150]:
def comaReplace(str):
    return (str.replace(",","").strip())


In [149]:
def getFirstNumber(str):
    repl_str = re.compile('(\d+[\,\d+])')
    match = re.search(repl_str, str)
    return float(match.group().replace(",","."))


In [148]:
def getNumberStr(str):
    repl_str = re.compile('(\d Zimmer)')
    match = re.search(repl_str, str)
    return float(match.group().replace(" Zimmer",""))



## Soup Object

In [ ]:
soup =BeautifulSoup(response.content, 'html.parser')
baseUrl="https://www.dibeo.at"
class RealEstate:    
    def __init__(self):
        self._terasseBalcony=""
        self._type=""
        self._name=""
        self._location=""
        self._price=""
        self._contractType=""
        self._livingarea=""
        self._rooms=""
        self._buildingtype=""
        self._objectType=""
        self._furnishing=""
    def set_type(self, value):
        self._type = value
    def set_name(self, value):
        self._name = value
    def set_location(self, value):
        self._location = value         
    def set_price(self, value):
        self._price = value
    #def set_provision(self, value):
        #self._provision = value
    def set_contractType(self, value):
        self._contractType = value
    def set_livingarea(self, value):
        self._livingarea = value
    def set_rooms(self, value):
        self._rooms = value
    def set_terasseBalcony(self, value):
        self._terasseBalcony = value
    def set_buildingtype(self, value):
        self._buildingtype = value
    def set_objectType(self, value):
        self._objectType = value
    def set_furnishing(self, value):
        self._furnishing = value

    def as_dict(self):
        return {'type': self._type, 'name': self._name, 'location': self._location, 
        'price': self._price, 'contractType': self._contractType, 'livingarea': self._livingarea, 
        'rooms': self._rooms, 'terasseBalcony': self._terasseBalcony, 'buildingtype': self._buildingtype, 
        'objectType': self._objectType, 'furnishing': self._furnishing}
    
# function for getering product data 
def getProductData(link):
    realEstate=RealEstate()    
    page = requests.get(baseUrl+link)
    soup = BeautifulSoup(page.content, "html.parser")
    productName = soup.find("h1")
    productLocation = soup.find("h2")
    realEstate.set_type("Apartment")
    realEstate.set_name(productName.getText())
    realEstate.set_location(productLocation.getText())
    productAttributes = soup.find_all("div", {"class": "attribute"})

    for productAttribute in productAttributes:
        productAttributeTitle=productAttribute.find("div", {"class": "attribute-title"}).getText()
        productAttributeValue=productAttribute.find("div", {"class": "attribute-value"}).getText()   
        if("Kosten" in productAttributeTitle):
            realEstate.set_price(getNumber(productAttributeValue))
        elif ("Vertragsart" in productAttributeTitle):
            realEstate.set_contractType(comaReplace(productAttributeValue))
        elif ("Wohnfläche" in productAttributeTitle):
            realEstate.set_livingarea(getFirstNumber(productAttributeValue))
            realEstate.set_rooms(getNumberStr(productAttributeValue))
        elif ("Freifläche" in productAttributeTitle):
            realEstate.set_terasseBalcony(comaReplace(productAttributeValue))
        elif ("Bauart" in productAttributeTitle):
            realEstate.set_buildingtype(comaReplace(productAttributeValue))
        print(productAttributeTitle+"="+productAttributeValue)
    return realEstate

products=[]
for page in range(417):
    response=getListOfRealEstateByPage(page)
    # harsing product links 
    subLinks = soup.find_all("a", {"class": "product"})
   
    for link in subLinks:    
        product=getProductData(link['href'])
        print(product)
        products.append(product)

df = pd.DataFrame([x.as_dict() for x in products])
df.to_csv('realestate.csv', index=False)

